In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import gutenberg, stopwords
from collections import Counter

In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ', text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [3]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en_core_web_sm')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [4]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

alice_sents = alice_sents
persuasion_sents = persuasion_sents

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
print(sentences.shape)
sentences.head()

(5318, 2)


,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


In [5]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(500)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (not token.is_punct and not token.is_stop and token.lemma_ in common_words)]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [6]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000


,timidly,seat,explain,fetch,thought,regret,join,lie,to,lyme,...,introduce,nearer,come,continue,visit,somebody,meet,animal,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


In [8]:
#add number of words in a sentence
sent_list = []
for sentence in word_counts['text_sentence']:
    n = len(sentence)
    sent_list.append(n)
word_counts['number_words'] = sent_list

#add amount of punctuation
punct_list = []
for sentence in word_counts['text_sentence']:
    n = 0
    for token in sentence:
        token.lemma_
        if token.is_punct:
            n += 1
    punct_list.append(n)
word_counts['punctuation'] = punct_list  

In [9]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))
print('\nTest CV Scores:', cross_val_score(rfc, X_train, y_train, cv=10))

Training set score: 0.990909090909091

Test set score: 0.887687969924812

Test CV Scores: [0.896875   0.91875    0.8875     0.86520376 0.89655172 0.90282132
 0.87147335 0.86477987 0.88993711 0.89937107]


In [10]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))
print('\nTest CV Scores:', cross_val_score(lr, X_train, y_train, cv=10))

(3190, 776) (3190,)
Training set score: 0.9413793103448276

Test set score: 0.9172932330827067

Test CV Scores: [0.903125   0.903125   0.89375    0.89341693 0.92476489 0.94670846
 0.89341693 0.8836478  0.87735849 0.90251572]


In [11]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))
print('\nTest CV Scores:', cross_val_score(clf, X_train, y_train, cv=10))

Training set score: 0.8890282131661442

Test set score: 0.8731203007518797

Test CV Scores: [0.8625     0.846875   0.853125   0.85893417 0.88401254 0.91222571
 0.89028213 0.85849057 0.87735849 0.86792453]


In [12]:
from sklearn import svm

svc = svm.SVC()
svc.fit(X_train, y_train)
print('Training set score:', svc.score(X_train, y_train))
print('\nTest set score:', svc.score(X_test, y_test))
print('\nTest CV Scores:', cross_val_score(svc, X_train, y_train, cv=10))

Training set score: 0.7495297805642633

Test set score: 0.7598684210526315

Test CV Scores: [0.74375    0.740625   0.7625     0.71159875 0.7523511  0.7492163
 0.7523511  0.73584906 0.74528302 0.74213836]


In [13]:
from sklearn.naive_bayes import BernoulliNB

bnb = BernoulliNB()
bnb.fit(X_train, y_train)
print('Training set score:', bnb.score(X_train, y_train))
print('\nTest set score:', bnb.score(X_test, y_test))
print('\nTest CV Scores:', cross_val_score(bnb, X_train, y_train, cv=10))

Training set score: 0.9269592476489028

Test set score: 0.9229323308270677

Test CV Scores: [0.928125   0.903125   0.940625   0.88714734 0.91222571 0.93730408
 0.9184953  0.89308176 0.91823899 0.91194969]


In [14]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma)
print(emma[0:200])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to unite some of the best blessings of existence; and had lived nearly twenty-one years in the world w


In [15]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [16]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

# Emma is quite long, let's cut it down to the same length as Alice.
emma_sents = emma_sents[0:len(alice_sents)]

In [17]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500


In [18]:
#add number of words in a sentence
sent_list = []
for sentence in emma_bow['text_sentence']:
    n = len(sentence)
    sent_list.append(n)
emma_bow['number_words'] = sent_list

#add amount of punctuation
punct_list = []
for sentence in emma_bow['text_sentence']:
    n = 0
    for token in sentence:
        token.lemma_
        if token.is_punct:
            n += 1
    punct_list.append(n)
emma_bow['punctuation'] = punct_list 

In [19]:
# Now we can model it! Let's use Bernoulli's

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', bnb.score(X_Emma_test, y_Emma_test))
bnb_Emma_predicted = bnb.predict(X_Emma_test)
pd.crosstab(y_Emma_test, bnb_Emma_predicted)


Test set score: 0.6547352721849367


col_0,Austen,Carroll
row_0,,
Austen,1400,269
Carroll,657,356


In [20]:
# Now we can model it! Let's use logistic regression

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.6879194630872483


col_0,Austen,Carroll
row_0,,
Austen,1543,126
Carroll,711,302


In [22]:
moby = gutenberg.raw('melville-moby_dick.txt')
moby = text_cleaner(moby)

In [25]:
moby_doc = nlp(moby[:900000])

In [26]:
# Group into sentences.
moby_sents = [[sent, "Melville"] for sent in persuasion_doc.sents]

# Emma is quite long, let's cut it down to the same length as Alice.
moby_sents = moby_sents[0:len(alice_sents)]

In [27]:
# Build a new Bag of Words data frame for moby word counts.
# We'll use the same common words from Alice and Persuasion.
moby_sentences = pd.DataFrame(emma_sents)
moby_bow = bow_features(emma_sentences, common_words)

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500


In [28]:
#add number of words in a sentence
sent_list = []
for sentence in moby_bow['text_sentence']:
    n = len(sentence)
    sent_list.append(n)
moby_bow['number_words'] = sent_list

#add amount of punctuation
punct_list = []
for sentence in moby_bow['text_sentence']:
    n = 0
    for token in sentence:
        token.lemma_
        if token.is_punct:
            n += 1
    punct_list.append(n)
moby_bow['punctuation'] = punct_list 

In [29]:
# Combine the Emma sentence data with the Alice data from the test set.
X_moby_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    moby_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_moby_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * moby_bow.shape[0])])

# Model.
print('\nTest set score:', bnb.score(X_moby_test, y_moby_test))
bnb_moby_predicted = bnb.predict(X_moby_test)
pd.crosstab(y_moby_test, bnb_moby_predicted)


Test set score: 0.6547352721849367


col_0,Austen,Carroll
row_0,,
Austen,1400,269
Carroll,657,356
